Mermaid(
    """
    graph TB
    n1(["00 - Idle"])
    n2(["01 - Recirculating th. storage"])
    n3(["10 - Heating up solar field"])
    n4(["11 - Solar field heating th. storage"])

    n1 <--> n3
    n1 <--> n2
    n3 <--> n4
    n4 --> n1
    """
)


In [1]:
from typing import Literal
from enum import Enum
import time
from pathlib import Path
import numpy as np
import pandas as pd
import copy

from transitions.extensions import GraphMachine as Machine
import transitions as tr
from IPython.display import Image, Markdown
from loguru import logger

from solarmed_modeling.utils.fsms import test_state
from phd_visualizations import save_figure

%load_ext autoreload
%autoreload 2

output_path = Path("results")

valid_input: float = 1.0
invalid_input: float = 0.0



/workspaces/solarMED-modeling/.venv/lib/python3.11/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_params" in SolarMED has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


### Test the SFTS class

In [9]:
# Either use the class defined here
# Or import it from the package

from solarmed_modeling.fsms.sfts import SolarFieldWithThermalStorageFsm, FsmParameters, FsmInternalState
from solarmed_modeling.fsms import SfTsState

logger.enable("solarmed_modeling.fsms")

# Here we are testing:
# - the correct reset of the FSM 
# - change of fsm behaviours (elapsed samples) by modifying its params 
# - partial initialization suuport (effect also in elapsed samples) by modifying initial internal states
# - cooldown of states

sleep_time: int = 0
sample_time: int = 1
initial_state = SfTsState.IDLE

params_to_test: list[FsmParameters] = [

    FsmParameters(
        recirculating_ts_cooldown_time = 9999,
        idle_cooldown_time = 5,
    ),
    FsmParameters(
        recirculating_ts_cooldown_time = 3,
        idle_cooldown_time = 5,
    ),
    FsmParameters(
        recirculating_ts_enabled=False,
        recirculating_ts_cooldown_time = 0, # Indifferent
        idle_cooldown_time = 5,
    ),
]

initial_internal_state_to_test: list[FsmInternalState] = [
    FsmInternalState(
        idle_cooldown_done = False,
        recirculating_ts_cooldown_done = False
    ),
]


In [10]:
dfs: list[list[pd.DataFrame]] = [[pd.DataFrame()]*len(initial_internal_state_to_test) for _ in range(len(params_to_test))]
models: list[list[SolarFieldWithThermalStorageFsm]] = [[None]*len(initial_internal_state_to_test) for _ in range(len(params_to_test))]

for param_idx, params in enumerate(params_to_test):
    for init_int_st_idx, initial_internal_state in enumerate(initial_internal_state_to_test):
        logger.info(f"Testing alternative {param_idx+1}/{len(params_to_test)} for parameters and {init_int_st_idx+1}/{len(initial_internal_state_to_test)} for initial internal state. Each option is tested twice to validate FSM reset")
        logger.debug(f"{params=}, {initial_internal_state=}")
        
    # params = params_to_test[1]
    # initial_internal_state = initial_internal_state_to_test[0]        
        model = SolarFieldWithThermalStorageFsm(
            name="SFTS_FSM",
            sample_time=sample_time,
            initial_state=initial_state,
            params=params,
            internal_state=initial_internal_state,
            
            qsf=invalid_input, qts_src=invalid_input
        )
        
        for _ in range(1): # Run each option N times to verify reset works
            models[param_idx][init_int_st_idx] = copy.deepcopy(model)
            dfs[param_idx][init_int_st_idx] = model.to_dataframe(dfs[param_idx][init_int_st_idx])
               
            test_state(expected_state=initial_state.name, model=model)

            # Go from IDLE to ACTIVE
            dst_state = SfTsState.SF_HEATING_TS
            while model.state != dst_state:
                dfs[param_idx][init_int_st_idx] = model.step(
                    qsf=valid_input, qts_src=valid_input,
                    return_df=True, df=dfs[param_idx][init_int_st_idx]
                )
                
                logger.info(f"[Sample {model.current_sample}] Current state: {model.state.name}")
                time.sleep(sleep_time)
            test_state(expected_state=dst_state, model=model)
            
            # Go from ACTIVE to IDLE
            dst_state = SfTsState.IDLE
            while model.state != dst_state:
                dfs[param_idx][init_int_st_idx] = model.step(
                    qsf=invalid_input, qts_src=invalid_input,
                    return_df=True, df=dfs[param_idx][init_int_st_idx]
                )
                
                logger.info(f"[Sample {model.current_sample}] Current state: {model.state.name}")
                time.sleep(sleep_time)
            test_state(expected_state=dst_state, model=model)
            
            # Go from IDLE to ACTIVE
            dst_state = SfTsState.SF_HEATING_TS
            while model.state != dst_state:
                dfs[param_idx][init_int_st_idx] = model.step(
                    qsf=valid_input, qts_src=valid_input,
                    return_df=True, df=dfs[param_idx][init_int_st_idx]
                )
                
                logger.info(f"[Sample {model.current_sample}] Current state: {model.state.name}")
                time.sleep(sleep_time)
            test_state(expected_state=dst_state, model=model)
            
            # TODO: Add going from active to recirculating ts and getting stuck 
            # there for some samples before giving up
            dfs[param_idx][init_int_st_idx] = model.step(
                qsf=invalid_input, qts_src=invalid_input,
                return_df=True, df=dfs[param_idx][init_int_st_idx]
            )
            patience: int = 10
            for _ in range(patience):
                dfs[param_idx][init_int_st_idx] = model.step(
                    qsf=invalid_input, qts_src=valid_input,
                    return_df=True, df=dfs[param_idx][init_int_st_idx]
                )
            expected_state = SfTsState.RECIRCULATING_TS if (model.params.recirculating_ts_cooldown_time <= patience and 
                                                            model.params.recirculating_ts_enabled) else SfTsState.IDLE
            test_state(expected_state=expected_state, model=model)
            # # Go to IDLE - ACTIVE -> if we go to IDLE, it should take `active_cooldown_time` additional samples to be able to go back to ACTIVE
            # while model.state != MedState.IDLE:
            #     dfs[param_idx][init_int_st_idx] = model.step(
            #         qmed_s=invalid_input, qmed_f=valid_input, Tmed_s_in=valid_input, Tmed_c_out=valid_input, med_vacuum_state=MedVacuumState.LOW,
            #         return_df=True, df=dfs[param_idx][init_int_st_idx]
            #     )
            #     logger.info(f"[Sample {model.current_sample}] Current state: {model.state.name}")
            #     time.sleep(sleep_time)
            # test_state(expected_state='IDLE', model=model)
            # while model.state != MedState.ACTIVE:
            #     dfs[param_idx][init_int_st_idx] = model.step(
            #         qmed_s=valid_input, qmed_f=valid_input, Tmed_s_in=valid_input, Tmed_c_out=valid_input, med_vacuum_state=MedVacuumState.LOW,
            #         return_df=True, df=dfs[param_idx][init_int_st_idx]
            #     )
            #     logger.info(f"[Sample {model.current_sample}] Current state: {model.state.name}")
            #     time.sleep(sleep_time)
            # test_state(expected_state='ACTIVE', model=model)
            

            logger.info(f"It took {model.current_sample} samples to go from {initial_state.name} to {model.state.name}")

            # Reset the model back to the initial and initial internal states
            model.reset_fsm()

            assert model.current_sample == 0, "Model was not reset properly"
            assert model.state == initial_state, "Model was not reset properly"
            


2024-11-08 20:04:11.628 | INFO     | __main__:<module>:6 - Testing alternative 1/3 for parameters and 1/1 for initial internal state. Each option is tested twice to validate FSM reset
2024-11-08 20:04:11.629 | DEBUG    | __main__:<module>:7 - params=FsmParameters(recirculating_ts_enabled=True, recirculating_ts_cooldown_time=9999, idle_cooldown_time=5, startup_conditions=FsmStartupConditions(qsf=0.372, qts_src=0.95), shutdown_conditions=FsmShutdownConditions(qsf=0.0, qts_src=0.0)), initial_internal_state=FsmInternalState(idle_cooldown_done=False, idle_cooldown_elapsed_samples=0, recirculating_ts_cooldown_done=False, recirculating_ts_cooldown_elapsed_samples=0)
2024-11-08 20:04:11.632 | INFO     | solarmed_modeling.fsms:check_elapsed_samples:278 - [SFTS_FSM] Still idle cooldown, 1/5 to complete
2024-11-08 20:04:11.633 | INFO     | solarmed_modeling.fsms:check_elapsed_samples:278 - [SFTS_FSM] Still recirculating_ts cooldown, 1/9999 to complete
2024-11-08 20:04:11.635 | INFO     | __main__

2024-11-08 20:04:11.753 | INFO     | __main__:<module>:59 - [Sample 9] Current state: IDLE
2024-11-08 20:04:11.754 | INFO     | solarmed_modeling.fsms:check_elapsed_samples:278 - [SFTS_FSM] Still idle cooldown, 3/5 to complete
2024-11-08 20:04:11.754 | DEBUG    | solarmed_modeling.fsms:get_next_valid_transition:252 - [SFTS_FSM] No transition found and inputs are the same as in prior iteration, staying in the same state
2024-11-08 20:04:11.756 | INFO     | __main__:<module>:59 - [Sample 10] Current state: IDLE
2024-11-08 20:04:11.757 | INFO     | solarmed_modeling.fsms:check_elapsed_samples:278 - [SFTS_FSM] Still idle cooldown, 4/5 to complete
2024-11-08 20:04:11.757 | DEBUG    | solarmed_modeling.fsms:get_next_valid_transition:252 - [SFTS_FSM] No transition found and inputs are the same as in prior iteration, staying in the same state
2024-11-08 20:04:11.758 | INFO     | __main__:<module>:59 - [Sample 11] Current state: IDLE
2024-11-08 20:04:11.760 | INFO     | solarmed_modeling.fsms:c

### Visualize state evolution of tests

In [11]:
from solarmed_modeling.visualization.fsm.state_evolution import plot_episode_state_evolution

save_figures: bool = True

for param_idx in range(len(params_to_test)):
    for init_int_st_idx in range(len(initial_internal_state_to_test)):
        df = dfs[param_idx][init_int_st_idx]
        model = models[param_idx][init_int_st_idx]
        fig = plot_episode_state_evolution(model=model, df=df)
        
        if save_figures:
            save_figure(
                fig, figure_name=f"{model.name}_test_state_evolution_{param_idx}{init_int_st_idx}", figure_path=output_path, formats=["html", "png"]
            )

        fig.show()


2024-11-08 20:04:15.336 | INFO     | phd_visualizations:save_figure:38 - Figure saved in [PosixPath('results')]/SFTS_FSM_test_state_evolution_00.html


2024-11-08 20:04:15.512 | INFO     | phd_visualizations:save_figure:38 - Figure saved in [PosixPath('results')]/SFTS_FSM_test_state_evolution_00.png


2024-11-08 20:04:15.576 | INFO     | phd_visualizations:save_figure:38 - Figure saved in [PosixPath('results')]/SFTS_FSM_test_state_evolution_10.html
2024-11-08 20:04:15.716 | INFO     | phd_visualizations:save_figure:38 - Figure saved in [PosixPath('results')]/SFTS_FSM_test_state_evolution_10.png


2024-11-08 20:04:15.871 | INFO     | phd_visualizations:save_figure:38 - Figure saved in [PosixPath('results')]/SFTS_FSM_test_state_evolution_20.html
2024-11-08 20:04:16.012 | INFO     | phd_visualizations:save_figure:38 - Figure saved in [PosixPath('results')]/SFTS_FSM_test_state_evolution_20.png
